# CPE 490 Final Project
### By Elijah Moore  

## 1. Acquiring the Data

In [2]:
import yfinance as yf
import datetime as dt
import pandas as pd

# Stock Tickers
apple = "AAPL"
tesla = "TSLA"
microsoft = "MSFT"
nvidia = "NVDA"
ford = "F"
google = "GOOGL"


# Acquires two years of Historical Data
start_date = dt.datetime.today()- dt.timedelta(days=730) 
end_date = dt.datetime.today()

apple_hist = yf.download(apple, start_date, end_date)
tesla_hist = yf.download(tesla, start_date, end_date)
microsoft_hist = yf.download(microsoft, start_date, end_date)
nvidia_hist = yf.download(nvidia, start_date, end_date)
ford_hist = yf.download(ford, start_date, end_date)
google_hist = yf.download(google, start_date, end_date)


apple_hist.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-04-15,98.441002,98.683998,97.623001,98.405998,98.405998,0
2013-04-16,96.801003,98.151001,96.588997,96.800003,96.800003,0
2013-04-17,97.607002,98.419998,97.524002,97.610001,97.610001,0
2013-04-18,98.059998,98.501999,97.639999,98.028999,98.028999,0
2013-04-19,98.222000,99.329002,98.114998,98.203003,98.203003,0
